In [1]:
import cv2
import time
from pydarknet import Detector, Image
from tqdm import tqdm

In [2]:
net = Detector(bytes("cfg/yolov3.cfg", encoding="utf-8"), bytes("yolov3.weights", encoding="utf-8"), 0, bytes("cfg/coco.data",encoding="utf-8"))
tracker_type = "medianflow"
tracker = cv2.TrackerMedianFlow_create

In [3]:
def dawhtsal(bbox1, bbox2):
    #2 hairtsagnii dawhtsaliig tootsoolno.
    bbox1_x1 = bbox1[0]
    bbox1_y1 = bbox1[1]
    bbox1_x2 = bbox1[0] + bbox1[2]
    bbox1_y2 = bbox1[1] + bbox1[3]
    bbox2_x1 = bbox2[0]
    bbox2_y1 = bbox2[1]
    bbox2_x2 = bbox2[0] + bbox2[2]
    bbox2_y2 = bbox2[1] + bbox2[3]
    dawhtsal_x1 = max(bbox1_x1, bbox2_x1)
    dawhtsal_y1 = max(bbox1_y1, bbox2_y1)
    dawhtsal_x2 = min(bbox1_x2, bbox2_x2)
    dawhtsal_y2 = min(bbox1_y2, bbox2_y2)
    dawhtsal_width = dawhtsal_x2 - dawhtsal_x1
    dawhtsal_height = dawhtsal_y2 - dawhtsal_y1
    if dawhtsal_width < 0 or dawhtsal_height < 0:
        return 0.0
    dawhtsal_area = dawhtsal_width * dawhtsal_height
    bbox1_area = (bbox1_x2 - bbox1_x1) * (bbox1_y2 - bbox1_y1)
    bbox2_area = (bbox2_x2 - bbox2_x1) * (bbox2_y2 - bbox2_y1)
    if bbox1_area < bbox2_area:
        jijig_area = bbox1_area  
    else:
        jijig_area = bbox2_area
    too = 0.00001 # toog 0d huwaagdahaas sergiilew
    dawhtsal_huwi = dawhtsal_area / (jijig_area + too) 
    return dawhtsal_huwi
# bbox1 = [0,0,100,100]
# bbox2 = [99,99,100,100]

In [4]:
def _line_segments_intersect(line1, line2):
    def get_orientation(p, q, r):
        val = (q[1] - p[1]) * (r[0] - q[0]) - (q[0] - p[0]) * (r[1] - q[1])
        if val == 0:
            return 0
        return 1 if val > 0 else 2
    def is_on_segment(p, q, r):
        if q[0] <= max(p[0], r[0]) and q[0] >= min(p[0], r[0]) and \
            q[1] <= max(p[1], r[1]) and q[1] >= min(p[1], r[1]):
            return True
        return False
    p1 = line1[0]
    q1 = line1[1]
    p2 = line2[0]
    q2 = line2[1]
    o1 = get_orientation(p1, q1, p2)
    o2 = get_orientation(p1, q1, q2)
    o3 = get_orientation(p2, q2, p1)
    o4 = get_orientation(p2, q2, q1)
    if o1 != o2 and o3 != o4:
        return True

    if o1 == 0 and is_on_segment(p1, p2, q1):
        return True

    if o2 == 0 and is_on_segment(p1, q2, q1):
        return True

    if o3 == 0 and is_on_segment(p2, p1, q2):
        return True

    if o4 == 0 and is_on_segment(p2, q1, q2):
        return True

    return False

def _has_crossed_counting_line(bbox, line):
   
    x, y, w, h = bbox
    bbox_line1 = [(x, y), (x + w, y)]
    bbox_line2 = [(x + w, y), (x + w, y + h)]
    bbox_line3 = [(x, y), (x, y + h)]
    bbox_line4 = [(x, y + h), (x + w, y + h)]
    if _line_segments_intersect(bbox_line1, line) or \
            _line_segments_intersect(bbox_line2, line) or \
            _line_segments_intersect(bbox_line3, line) or \
            _line_segments_intersect(bbox_line4, line):
        return True
    return False

In [5]:
def video_unshih(path):
    video = cv2.VideoCapture(path)
    width  = video.get(3)  # float
    height = video.get(4) # float
    if not video.isOpened():
        return None
    ok, first_frame = video.read()
    if not ok:
        return None
    return video,first_frame,width,height

In [6]:
def click(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        refPt.append((x,y))

In [7]:
def mashin_detection(frame,bbox,mashin_trackers,tracker):
    zurag = Image(frame)
    results = net.detect(zurag)
    for cat, score, bounds in results:
        temp = str(cat.decode("utf-8"))
        if not temp == "car" and not temp == "truck" and not temp == "bus":
            continue
        elif score<0.5:
            continue
        x, y, w, h = bounds
        bbox1 = (int(x - w / 2),int(y - h / 2),w,h)
        temp = bbox
        for j in temp:
            if dawhtsal(bbox1, j) < 0.6:
                bbox.append(bbox1)
                mashin_trackers.append([tracker(),0,None,None])
    return bbox,mashin_trackers

In [8]:
temp = video_unshih("-bb2.mp4")
if temp:
    video,frame,width,height = temp
else:
    print("Videog unshij chadsangui")

In [9]:
# # Line zurah
# refPt = []
# image = frame
# clone = image.copy()
# cv2.namedWindow("Startline")
# cv2.setMouseCallback("Startline", click)
# start = []
# cnt=0
# while cnt<4:
#     clone2 = image.copy()
#     refPt = []
#     flag = 0
#     while True:
#         cv2.imshow("Startline", image)
#         key = cv2.waitKey(1) & 0xFF
#         if key == ord("r"):
#             image = clone.copy()
#             if flag == 0:
#                 start.pop()
#                 flag = 1
#                 cnt-=1
#         elif key == ord("c"):
#             break
#         if len(refPt) == 2:
#             color = (0, 255, 0)  
#             thickness = 6
#             image = cv2.line(image, refPt[0] , refPt[1] , color, thickness)
#             break
#     start.append(refPt)
#     if cnt == 3:
#         refPt = []
#         while True:
#             cv2.imshow("Startline", image)
#             key = cv2.waitKey(1) & 0xFF
#             if key == ord("r"):
#                 image = clone.copy()
#                 if flag == 0:
#                     start.pop()
#                     flag = 1
#             elif key == ord("c"):
#                 break
#             if len(refPt) == 2:
#                 color = (0, 255, 0)  
#                 thickness = 6
#                 image = cv2.line(image, refPt[0] , refPt[1] , color, thickness)
#                 break
#     clone = clone2.copy()
#     cnt+=1
# cv2.destroyAllWindows()
# cv2.namedWindow("Endline")
# cv2.setMouseCallback("Endline", click)
# clone = image.copy()
# end = []
# cnt = 0
# while cnt<4:
#     clone2 = image.copy()
#     refPt = []
#     flag = 0
#     while True:
#         cv2.imshow("Endline", image)
#         key = cv2.waitKey(1) & 0xFF
#         if key == ord("r"):
#             image = clone.copy()
#             if flag == 0:
#                 end.pop()
#                 flag = 1
#         elif key == ord("c"):
#             break
#         if len(refPt) == 2:
#             color = (0, 0, 255)  
#             thickness = 6
#             image = cv2.line(image, refPt[0] , refPt[1] , color, thickness)
#             break
#     end.append(refPt)
#     if cnt == 3:
#         refPt = []
#         while True:
#             cv2.imshow("Endline", image)
#             key = cv2.waitKey(1) & 0xFF
#             if key == ord("r"):
#                 image = clone.copy()
#                 if flag == 0:
#                     end.pop()
#                     flag = 1
#             elif key == ord("c"):
#                 break
#             if len(refPt) == 2:
#                 color = (0, 0, 255)  
#                 thickness = 6
#                 image = cv2.line(image, refPt[0] , refPt[1] , color, thickness)
#                 break
#     clone = clone2.copy()
#     cnt+=1
# cv2.destroyAllWindows()

In [10]:
# print(start,end)
# Tsag uur (start ,endline)
# start = [[(1383, 362), (1396, 375)], [(1378, 481), (1043, 471)], [(756, 366), (736, 397)], [(833, 283), (1099, 321)]]
# end = [[(1481, 376), (1483, 408)], [(701, 506), (934, 517)], [(746, 303), (734, 352)], [(1194, 273), (1263, 273)]]
# dulguun nuur (start ,endline)
start = [[(852, 338), (893, 409)], [(938, 567), (253, 556)], [(78, 384), (3, 428)], [(278, 308), (555, 321)]] 
end = [[(973, 391), (998, 449)], [(3, 448), (38, 531)], [(45, 330), (0, 368)], [(595, 280), (702, 272)]]

In [11]:
# cv2.imshow("e", frame)
# cv2.waitKey(10000)
# cv2.destroyAllWindows()

In [ ]:
start_time = time.time()
bbox=[]
mashin_trackers=[]
zurag = Image(frame)
results = net.detect(zurag)
for cat, score, bounds in results:
    temp = str(cat.decode("utf-8"))
    if not temp == 'car' and not temp == 'truck' and not temp == 'bus':
        continue
    elif score<0.5:
        continue
    x, y, w, h = bounds
    bbox.append((int(x - w / 2),int(y - h / 2),w,h)) 
    mashin_trackers.append([tracker(),0,None,None])
for i in range(len(bbox)):
    ok = mashin_trackers[i][0].init(frame, bbox[i])
    for k,j in enumerate(start):
        if _has_crossed_counting_line(bbox[i], j):
            mashin_trackers[i][2]=k
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('Garalt'+tracker_type+'.avi', fourcc, 20.0, (int(width),int(height)))
toologdson = []
frame_det = 1
pbar = tqdm(total=264)
while True:
    ok, frame = video.read()
    pbar.update(1)
    if not ok:
        out.release()
        break
    new_tracker=[]
    new_bbox=[]
    if frame_det == 30:
        bbox,mashin_trackers = mashin_detection(frame,bbox,mashin_trackers,tracker)
        frame_det=0
    for i in range(len(bbox)):
        ok, bbox[i] = mashin_trackers[i][0].update(frame)
        if mashin_trackers[i][2] is None:
            for k,j in enumerate(start):
                if _has_crossed_counting_line(bbox[i], j):
                    mashin_trackers[i][2]=k
        if mashin_trackers[i][2]:
            for k,j in enumerate(end):
                if _has_crossed_counting_line(bbox[i], j) and not _has_crossed_counting_line(bbox[i], start[mashin_trackers[i][2]]):
                    mashin_trackers[i][3]=k    
        if ok:
            p1 = (int(bbox[i][0]), int(bbox[i][1]))
            p2 = (int(bbox[i][0] + bbox[i][2]), int(bbox[i][1] + bbox[i][3]))
            cv2.rectangle(frame, p1, p2, (255,0,0), 4, 1)
        else:
            mashin_trackers[i][1]+=1
        if not mashin_trackers[i][3] is None and not mashin_trackers[i][2] is None:
            toologdson.append([mashin_trackers[i][2],mashin_trackers[i][3]])
            continue
        elif mashin_trackers[i][1]>=20:
            continue
        new_tracker.append(mashin_trackers[i])
        new_bbox.append(bbox[i])
    for i in end:
        start_point = i[0]
        end_point = i[1]
        color = (0, 0, 255)  
        thickness = 6
        frame = cv2.line(frame, start_point, end_point, color, thickness)
    for i in start:
        start_point = i[0]
        end_point = i[1]
        color = (0, 255, 0)  
        thickness = 6
        frame = cv2.line(frame, start_point, end_point, color, thickness)
    out.write(frame)
    mashin_trackers = new_tracker
    bbox = new_bbox 
    frame_det+=1
end_time = time.time()
pbar.close()
print(end_time-start_time)

 11%|█▏        | 30/264 [00:09<01:09,  3.37it/s]

In [5]:
zug = {"0":"Hoid","1":"Zuun","2":"Urd","3":"Baruun"}
tooloh = {}
for i in range(4):
    tooloh[zug[str(i)]]={}
for i in toologdson:
    start_zug = zug[str(i[0])]
    end_zug = zug[str(i[1])]
    
    if tooloh.get(start_zug).get(end_zug):
        tooloh[start_zug][end_zug]+=1
    else:
        tooloh[start_zug][end_zug]=1
for key in tooloh:
    for i in tooloh[key]:
        print(str(key)+"--->"+str(i)+" : "+str(tooloh[key][i]))

Zuun--->Baruun : 1
Urd--->Zuun : 1
Urd--->Urd : 1
Baruun--->Zuun : 3
Baruun--->Urd : 1
Baruun--->Baruun : 2
